In [1]:
import os
import json
import time
import torch
import argparse
import numpy as np
from multiprocessing import cpu_count
from torch.utils.data import DataLoader
from collections import OrderedDict, defaultdict
import torch.nn as nn
from torch.autograd import Variable

from ptb import PTB
from utils import to_var, idx2word, expierment_name
from model import SentenceVAE

In [2]:
ts = time.strftime('%Y-%b-%d-%H-%M-%S', time.gmtime())

In [3]:
save_model_path = os.path.join('won', ts)
os.makedirs('.\\'+save_model_path)

In [4]:
datasets = OrderedDict()

In [5]:
splits = ['train', 'valid'] + (['test'] if False else [])

In [6]:
splits

['train', 'valid']

In [7]:
for split in splits:
    datasets[split] = PTB(
        data_dir='data',
        split=split,
        create_data=False,
        max_sequence_length=60,
        min_occ=1
    )

In [8]:
vocab_size = datasets['train'].vocab_size

In [9]:
sos_idx=datasets['train'].sos_idx

In [10]:
eos_idx=datasets['train'].eos_idx

In [11]:
pad_idx=datasets['train'].pad_idx

In [12]:
max_sequence_length = 60
embedding_size = 300
hidden_size = 256
word_dropout = 0.5
latent_size = 16
num_layers = 1
bidirectional = True
batch_size_fit = 32
rnn_type = 'gru'
learning_rate = 0.0001
k = 0.0025
x0 = 2500

In [13]:
class RVAE(nn.Module):
    def __init__(self,vocab_size, embedding_size, max_sequence_length, hidden_size, word_dropout, latent_size,
                sos_idx, eos_idx, pad_idx, rnn_type='rnn' , num_layers=1, bidirectional=True):
        
        super().__init__()
        
        self.max_sequence_length = max_sequence_length
        self.sos_idx = sos_idx
        self.eos_idx = eos_idx
        self.pad_idx = pad_idx
        
        self.latent_size = latent_size
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        
        #self.outputs2vocab = nn.Linear(hidden_size * (2 if bidirectional else 1), vocab_size)
        self.encoder = Encoder(vocab_size = vocab_size,embedding_size = embedding_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional = bidirectional,latent_size = latent_size,rnn_type = rnn_type).cuda()
        self.decoder = Decoder(vocab_size = vocab_size,embedding_size = embedding_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional = bidirectional,latent_size = latent_size,rnn_type = rnn_type).cuda()
    
    def forward(self,x,length):
        
        mu,logvar,reparam = self.encoder(Variable(x),length)
        logp  = self.decoder(Variable(x),reparam)
        
        
        return logp, mu, logvar, reparam

In [14]:
class Encoder(nn.Module):
    def __init__(self,vocab_size,embedding_size, hidden_size, latent_size, bidirectional=True, num_layers = 1,rnn_type='rnn'):
        super(Encoder,self).__init__()
        
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.latent_size = latent_size
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        if self.rnn_type == 'rnn':
            rnn = nn.RNN
        elif self.rnn_type == 'gru':
            rnn = nn.GRU
        elif self.rnn_type =='lstm':
            rnn = nn.LSTM
        else:
            raise ValueError()
            
        self.embedding = nn.Embedding(vocab_size,embedding_size)
        self.encoder = rnn(self.embedding_size, self.hidden_size, num_layers = self.num_layers, bidirectional = self.bidirectional, batch_first = True)
        
        self.hidden_factor = (2 if self.bidirectional else 1) * self.num_layers
        
        self.hidden2mean = nn.Linear(self.hidden_size* self.hidden_factor, self.latent_size)
        self.hidden2logv = nn.Linear(self.hidden_size* self.hidden_factor, self.latent_size)
                        
    
    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        
        eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps).cuda()
        
        return eps.mul(std).add_(mu)
    
    def forward(self,x,length):
        batch_size = x.size(0)
        sorted_lengths, sorted_idx = torch.sort(length, descending=True)
        input_sequence = x[sorted_idx.cuda()]
        #print(input_sequence)
        _,hidden = self.encoder(self.embedding(input_sequence))
        if self.bidirectional or self.num_layers > 1:
            # flatten hidden state
            hidden = hidden.view(batch_size, self.hidden_size*self.hidden_factor)
        else:
            hidden = hidden.squeeze()

        mu = self.hidden2mean(hidden)
        logvar = self.hidden2logv(hidden)

        reparam = self.reparametrize(mu,logvar)
        
        return mu,logvar,reparam
        


In [15]:
class Decoder(nn.Module):
    def __init__(self,vocab_size,embedding_size, hidden_size, latent_size, bidirectional=True, num_layers = 1,rnn_type='rnn',word_dropout = 0.5):
        super(Decoder,self).__init__()
        
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.latent_size = latent_size
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        if self.rnn_type == 'rnn':
            rnn = nn.RNN
        elif self.rnn_type == 'gru':
            rnn = nn.GRU
        elif self.rnn_type =='lstm':
            rnn = nn.LSTM
        else:
            raise ValueError()
            
        self.hidden_factor = (2 if self.bidirectional else 1) * self.num_layers            
        self.latent2hidden = nn.Linear(latent_size, hidden_size * self.hidden_factor)
        
        self.embedding = nn.Embedding(vocab_size,embedding_size)
        self.word_dropout = nn.Dropout(p=word_dropout)
        
        self.decoder = rnn(embedding_size, hidden_size, num_layers=num_layers, bidirectional=self.bidirectional, batch_first=True)
        self.outputs2vocab = nn.Linear(hidden_size * (2 if bidirectional else 1), vocab_size)
        
    def forward(self,x,z):
        
        batch_size = x.size(0)
        sorted_lengths, sorted_idx = torch.sort(length, descending=True)
        input_sequence = x[sorted_idx.cuda()]
        
        hidden = self.latent2hidden(z)
        if self.bidirectional or self.num_layers > 1:
            # unflatten hidden state
            hidden = hidden.view(self.hidden_factor, batch_size, self.hidden_size)
        else:
            hidden = hidden.unsqueeze(0)
            
        outputs,_ = self.decoder(self.embedding(input_sequence),hidden)
        
        logp =nn.functional.log_softmax(self.outputs2vocab(outputs))
        
        return logp
        


In [17]:
rvae=RVAE(vocab_size, embedding_size, max_sequence_length, hidden_size, word_dropout, latent_size,sos_idx, eos_idx, pad_idx , rnn_type='gru')

In [18]:
def kl_anneal_function(anneal_function, step, k, x0):
    if anneal_function == 'logistic':
        return float(1/(1+np.exp(-k*(step-x0))))
    elif anneal_function == 'linear':
        return min(1, step/x0)

NLL = torch.nn.NLLLoss(size_average=False, ignore_index=datasets['train'].pad_idx)

In [19]:
def loss_fn(logp, target, length, mean, logv, anneal_function, step, k, x0):

    # cut-off unnecessary padding from target, and flatten
    target = target[:, :60].contiguous().view(-1)
    logp = logp.view(-1, logp.size(2))

    # Negative Log Likelihood
    NLL_loss = NLL(logp, target)

    # KL Divergence
    KL_loss = -0.5 * torch.sum(1 + logv - mean.pow(2) - logv.exp())
    KL_weight = kl_anneal_function(anneal_function, step, k, x0)

    return NLL_loss, KL_loss, KL_weight

In [20]:
optimizer = torch.optim.Adam(rvae.parameters(), lr=learning_rate)

In [ ]:
print(rvae),rvae.parameters()

In [22]:
epochs = 30

In [ ]:
step = 0
avg_losses = []
losses = []
NLL_losses = []
KL_losses = []
KL_weights = []

for epoch in range(epochs):
    for split in splits:
        data_loader = DataLoader(
                    dataset=datasets[split],
                    batch_size=batch_size_fit,
                    shuffle=split=='train',
                    num_workers=cpu_count(),
                    pin_memory=torch.cuda.is_available()
                )

        if split == 'train':
            rvae.train()
        else:
            rvae.eval()
            
        for iteration, batch in enumerate(data_loader):
            batch_size = batch['input'].size(0)
            
            optimizer.zero_grad()
            
            x = batch['input'].type(torch.cuda.LongTensor)
            length = batch['length']
            
            logp, mean, logv, z=rvae(x,length)
            
            NLL_loss, KL_loss, KL_weight = loss_fn(logp, Variable(batch['target']).type(torch.cuda.LongTensor),batch['length'], mean, logv, 'logistic', step, k, x0)
            
            loss = (NLL_loss + KL_weight * KL_loss)/batch_size
            
            losses.append(float(loss.cpu().data))
            NLL_losses.append(NLL_loss.data[0]/batch_size)
            KL_losses.append(KL_loss.data[0]/batch_size)
            KL_weights.append(KL_weight)
            

            
            if split == 'train':
                loss.backward()
                optimizer.step()
                step += 1

            if iteration % 100 == 0 or iteration+1 == len(data_loader):
                print("%s Batch %04d/%i, Loss %9.4f, NLL-Loss %9.4f, KL-Loss %9.4f, KL-Weight %6.3f"
            %(split.upper(), iteration, len(data_loader)-1, loss.data[0], NLL_loss.data[0]/batch_size, KL_loss.data[0]/batch_size, KL_weight))
                np.savez(L=losses,file='loss.npz')
                np.savez(L=NLL_losses,file='NLL_losses.npz')
                np.savez(L=KL_losses,file='KL_losses.npz') 
                np.savez(L=KL_weights,file='KL_weights.npz')
                
         # save checkpoint
        if split == 'train':
            checkpoint_path = os.path.join(save_model_path, "E%i.pytorch"%(epoch))
            torch.save(rvae, checkpoint_path)
            print("Model saved at %s"%checkpoint_path)
        print("%s Epoch %02d/%i, Mean ELBO %9.4f"%(split.upper(), epoch, epochs, np.mean(np.array(losses))))
        avg_losses.append(np.mean(np.array(losses)))
        np.savez(L=avg_losses,file='avg_losses.npz')

C:\Users\asd13\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:46: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


TRAIN Batch 0000/1314, Loss   86.3271, NLL-Loss   86.3259, KL-Loss    0.6551, KL-Weight  0.002
TRAIN Batch 0100/1314, Loss   82.6631, NLL-Loss   82.6601, KL-Loss    1.1949, KL-Weight  0.002
TRAIN Batch 0200/1314, Loss   69.8017, NLL-Loss   69.7987, KL-Loss    0.9662, KL-Weight  0.003
TRAIN Batch 0300/1314, Loss   83.6196, NLL-Loss   83.6157, KL-Loss    0.9511, KL-Weight  0.004
TRAIN Batch 0400/1314, Loss   61.0029, NLL-Loss   60.9991, KL-Loss    0.7356, KL-Weight  0.005
TRAIN Batch 0500/1314, Loss   82.7436, NLL-Loss   82.7404, KL-Loss    0.4821, KL-Weight  0.007
TRAIN Batch 0600/1314, Loss   86.1542, NLL-Loss   86.1510, KL-Loss    0.3664, KL-Weight  0.009
TRAIN Batch 0700/1314, Loss   81.6193, NLL-Loss   81.6172, KL-Loss    0.1946, KL-Weight  0.011
TRAIN Batch 0800/1314, Loss   77.4407, NLL-Loss   77.4376, KL-Loss    0.2156, KL-Weight  0.014
TRAIN Batch 0900/1314, Loss   69.4104, NLL-Loss   69.4089, KL-Loss    0.0877, KL-Weight  0.018
TRAIN Batch 1000/1314, Loss   82.8656, NLL-Loss   

C:\Users\asd13\AppData\Local\Continuum\Anaconda3\lib\site-packages\torch\serialization.py:159: UserWarning: Couldn't retrieve source code for container of type RVAE. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\asd13\AppData\Local\Continuum\Anaconda3\lib\site-packages\torch\serialization.py:159: UserWarning: Couldn't retrieve source code for container of type Encoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\asd13\AppData\Local\Continuum\Anaconda3\lib\site-packages\torch\serialization.py:159: UserWarning: Couldn't retrieve source code for container of type Decoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model saved at won\2018-May-14-14-34-55\E0.pytorch
TRAIN Epoch 00/30, Mean ELBO   79.3321
VALID Batch 0000/105, Loss   81.2188, NLL-Loss   81.2173, KL-Loss    0.0298, KL-Weight  0.049
VALID Batch 0100/105, Loss   61.7908, NLL-Loss   61.7893, KL-Loss    0.0300, KL-Weight  0.049
VALID Batch 0105/105, Loss   52.7749, NLL-Loss   52.7735, KL-Loss    0.0287, KL-Weight  0.049
VALID Epoch 00/30, Mean ELBO   79.2570
TRAIN Batch 0000/1314, Loss   81.1420, NLL-Loss   81.1406, KL-Loss    0.0290, KL-Weight  0.049
TRAIN Batch 0100/1314, Loss   81.1610, NLL-Loss   81.1597, KL-Loss    0.0209, KL-Weight  0.062
TRAIN Batch 0200/1314, Loss   79.7862, NLL-Loss   79.7845, KL-Loss    0.0212, KL-Weight  0.079
TRAIN Batch 0300/1314, Loss   74.0273, NLL-Loss   74.0264, KL-Loss    0.0085, KL-Weight  0.099
TRAIN Batch 0400/1314, Loss   71.7093, NLL-Loss   71.7075, KL-Loss    0.0148, KL-Weight  0.123
TRAIN Batch 0500/1314, Loss   79.9919, NLL-Loss   79.9913, KL-Loss    0.0043, KL-Weight  0.153
TRAIN Batch 0600/13

TRAIN Batch 0500/1314, Loss   74.9350, NLL-Loss   74.9345, KL-Loss    0.0005, KL-Weight  1.000
TRAIN Batch 0600/1314, Loss   69.2503, NLL-Loss   69.2499, KL-Loss    0.0004, KL-Weight  1.000
TRAIN Batch 0700/1314, Loss   67.7519, NLL-Loss   67.7512, KL-Loss    0.0007, KL-Weight  1.000
TRAIN Batch 0800/1314, Loss   77.3283, NLL-Loss   77.3279, KL-Loss    0.0004, KL-Weight  1.000
TRAIN Batch 0900/1314, Loss   72.7877, NLL-Loss   72.7872, KL-Loss    0.0005, KL-Weight  1.000
TRAIN Batch 1000/1314, Loss   76.4553, NLL-Loss   76.4547, KL-Loss    0.0006, KL-Weight  1.000
TRAIN Batch 1100/1314, Loss   79.7351, NLL-Loss   79.7346, KL-Loss    0.0005, KL-Weight  1.000
TRAIN Batch 1200/1314, Loss   73.2041, NLL-Loss   73.2035, KL-Loss    0.0006, KL-Weight  1.000
TRAIN Batch 1300/1314, Loss   80.4982, NLL-Loss   80.4979, KL-Loss    0.0003, KL-Weight  1.000
TRAIN Batch 1314/1314, Loss   68.4657, NLL-Loss   68.4654, KL-Loss    0.0003, KL-Weight  1.000
Model saved at won\2018-May-14-14-34-55\E5.pytorch

TRAIN Batch 1314/1314, Loss   69.6674, NLL-Loss   69.6672, KL-Loss    0.0002, KL-Weight  1.000
Model saved at won\2018-May-14-14-34-55\E9.pytorch
TRAIN Epoch 09/30, Mean ELBO   79.1149
VALID Batch 0000/105, Loss   81.0042, NLL-Loss   81.0039, KL-Loss    0.0003, KL-Weight  1.000
VALID Batch 0100/105, Loss   61.6326, NLL-Loss   61.6324, KL-Loss    0.0003, KL-Weight  1.000
VALID Batch 0105/105, Loss   52.2989, NLL-Loss   52.2987, KL-Loss    0.0003, KL-Weight  1.000
VALID Epoch 09/30, Mean ELBO   79.1076
TRAIN Batch 0000/1314, Loss   88.8195, NLL-Loss   88.8192, KL-Loss    0.0003, KL-Weight  1.000
TRAIN Batch 0100/1314, Loss   76.6938, NLL-Loss   76.6937, KL-Loss    0.0001, KL-Weight  1.000
TRAIN Batch 0200/1314, Loss   81.9853, NLL-Loss   81.9851, KL-Loss    0.0002, KL-Weight  1.000
TRAIN Batch 0300/1314, Loss   80.6849, NLL-Loss   80.6847, KL-Loss    0.0002, KL-Weight  1.000
TRAIN Batch 0400/1314, Loss   73.4182, NLL-Loss   73.4178, KL-Loss    0.0004, KL-Weight  1.000
TRAIN Batch 0500/13

In [ ]:
checkpoint_path

In [ ]:
try:
    rvae = torch.load(checkpoint_path)
    print("\n--------model restored--------\n")
except:
    print("\n--------model not restored--------\n")
    pass

In [ ]:
rvae.cuda()
rvae.eval()

In [ ]:
def _save_sample( save_to, sample, running_seqs, t):
    # select only still running
    running_latest = save_to[running_seqs]
    #print(running_latest)
    # update token at position t
    running_latest[:,t] = sample.data
    #print(running_latest[:,t])
    # save back
    save_to[running_seqs] = running_latest

    return save_to

In [ ]:
batch_size = 1

In [ ]:
generations=torch.cuda.FloatTensor(batch_size, max_sequence_length).fill_(pad_idx).long()

In [ ]:
z = Variable(torch.randn([batch_size, latent_size])).cuda()

In [ ]:
hidden = rvae.decoder.latent2hidden(z)

In [ ]:
hidden = hidden.view(rvae.encoder.hidden_factor, batch_size, rvae.hidden_size)

In [ ]:
t = 0

while(t<max_sequence_length ):
    if t == 0:
        input_sequence = Variable(torch.Tensor(batch_size).fill_(sos_idx).long()).cuda()
    input_sequence = input_sequence.unsqueeze(1)
    input_embedding = rvae.encoder.embedding(input_sequence)
    output, hidden = rvae.decoder.decoder(input_embedding, hidden)
    logits = rvae.decoder.outputs2vocab(output)
    samples = torch.topk(logits,1,dim=-1)[1].squeeze()
    input_sequence = samples.squeeze()
    generations[:,t] = input_sequence.data
    if (input_sequence == eos_idx).cpu().data.numpy():
        break
    t+=1

In [ ]:
generations

In [ ]:
from utils import to_var, idx2word, interpolate

with open('data'+'/ptb.vocab.json', 'r') as file:
    vocab = json.load(file)

w2i, i2w = vocab['w2i'], vocab['i2w']

In [ ]:
samples = generations.cpu().numpy()

sent_str = [str()]*len(samples)

for i, sent in enumerate(samples):
    for word_id in sent:
        if word_id == w2i['<pad>']: 
            break
        sent_str[i] += i2w[str(word_id)] + " "
    sent_str[i] = sent_str[i].strip()

In [ ]:
print(''.join(sent_str))